# <center> Part 1: Extract data from Wikipedia page<center> 

### Questions

#### Start by creating a new Notebook for this assignment.

#### Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

#### To create the above dataframe:

1. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
1. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
1. More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
1. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
1. Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
1. In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

#### Submit a link to your Notebook on your Github repository.

In [1]:
!pip install BeautifulSoup4
!pip install requests

In [2]:
# Importing necessary libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

# Create the link and get the data from the link
wiki_link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wikidata = requests.get(wiki_link).text

soup = BeautifulSoup(wikidata,'xml')

print('Soup is ready')


Soup is ready


#### From the website, check the html code, we know that the table is in the class wikitable. So use the soup  function to find the table

In [3]:
data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    
    #Check the table, the first row displays the names of the columns. Use If else to separate the data in to data and column lists.
    
    if (index == 0):
        columns = section
    else:
        data.append(section)

#Transform the list into a Pandas DataFrame
df_can = pd.DataFrame(data = data,columns = columns)
df_can.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


#### Now, clean the data

In [4]:
# There are missing data "Not assigned". We can remove them
df_can.drop(df_can[df_can['Borough']=='Not assigned'].index, inplace=True)
df_can.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
-> This is the old table, now the table has "/" as the separator, so I will use the same "/" instead of the comma

In [5]:
df_can["Neighborhood"] = df_can.groupby("Postal code")["Neighborhood"].transform(lambda neigh: '/ '.join(neigh))
df_can.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [6]:
df_can['Neighborhood'].value_counts()

Downsview                          4
Willowdale                         2
Don Mills                          2
Garden District, Ryerson           1
Commerce Court / Victoria Hotel    1
                                  ..
Lawrence Park                      1
Regent Park / Harbourfront         1
Leaside                            1
Humber Summit                      1
Northwest                          1
Name: Neighborhood, Length: 98, dtype: int64

In [7]:
df_can.drop_duplicates()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
...,...,...,...
160,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,Business reply mail Processing CentrE
169,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...


In [8]:
df_can['Neighborhood'].replace("Not assigned", df_can["Borough"],inplace=True)
df_can.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [9]:
df_can.set_index('Postal code', inplace=True)
df_can.head()

,Borough,Neighborhood
Postal code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Regent Park / Harbourfront
M6A,North York,Lawrence Manor / Lawrence Heights
M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


#### In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [10]:
df_can.shape

(103, 2)

# <center>Part 2: Segmenting and Clustering Neighborhoods in Toronto<center>
    
    Use the link  http://cocl.us/Geospatial_data to get the data. Download it and load the file using pandas

In [27]:
lat_long_df=pd.read_csv('Geospatial_Coordinates.csv')
lat_long_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [28]:
lat_long_df.columns

Index(['Postal Code', 'Latitude', 'Longitude'], dtype='object')

In [29]:
#set the index to be the postal code
lat_long_df.rename(columns={'Postal Code':'Postal code'}, inplace=True)
lat_long_df.set_index('Postal code', inplace=True)
lat_long_df.head()

,Latitude,Longitude
Postal code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [30]:
df_can=df_can.join(lat_long_df)
df_can.head()

,Borough,Neighborhood,Latitude,Longitude
Postal code,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
